In [1]:
# Configure AWS profile for local development
%env AWS_PROFILE=platform-developer

env: AWS_PROFILE=platform-developer


In [ ]:
import pandas as pd

from adapters.axiell.runtime import AXIELL_CONFIG

# Load the Axiell window store (use_rest_api_table=True for production, False for local)
window_store = AXIELL_CONFIG.build_window_store(use_rest_api_table=True)

print("Window status table info:")
print(window_store.table)

Window status table info:
window_status(
  1: window_key: required string,
  2: window_start: required timestamptz,
  3: window_end: required timestamptz,
  4: state: required string,
  5: attempts: required int,
  6: last_error: optional string,
  7: updated_at: required timestamptz,
  8: record_ids: optional list<string>,
  9: tags: optional map<string, string>
),
partition by: [],
sort order: [],
snapshot: Operation.APPEND: id=6110261927768827163, parent_id=1339840846052478428, schema_id=0


## Example Queries

Query windows by state or time range

In [ ]:
# Query 1: Get all windows by state
failed_windows = window_store.list_by_state("failed")
print(f"Found {len(failed_windows)} failed windows")

if failed_windows:
    df_failed = pd.DataFrame(failed_windows)
    display(df_failed.head(10))

# Query 2: Get windows in a time range
from datetime import datetime, timezone

start_date = datetime(2024, 12, 1, tzinfo=timezone.utc)
end_date = datetime(2026, 12, 31, tzinfo=timezone.utc)

windows_in_range = window_store.list_in_range(start=start_date, end=end_date)
print(f"\nFound {len(windows_in_range)} windows between {start_date} and {end_date}")

if windows_in_range:
    df_range = pd.DataFrame(windows_in_range)
    display(df_range.head(10))

# Query 3: Load all window statuses
all_statuses = window_store.load_status_map()
print(f"\nTotal windows in store: {len(all_statuses)}")

# Show a sample
if all_statuses:
    sample_key = next(iter(all_statuses))
    print(f"\nSample window record (key: {sample_key}):")
    print(all_statuses[sample_key])

## Retrieve a Single Window by Key

Get details for a specific window using its key

In [ ]:
# Retrieve a specific window by key
window_key = "your_window_key_here"  # Replace with your window key

from pyiceberg.expressions import EqualTo

# Query the table for the specific window key
scan = window_store.table.scan().filter(EqualTo("window_key", window_key))
arrow_table = scan.to_arrow()

if arrow_table is not None and arrow_table.num_rows > 0:
    # Convert to list of dicts
    rows = arrow_table.to_pylist()
    window_record = rows[0]  # Should only be one match
    df_single = pd.DataFrame([window_record])
    display(df_single)
else:
    print(f"No window found with key: {window_key}")

## Delete a Window by Key

⚠️ **Warning**: This will permanently delete the window status record!

In [ ]:
# Delete by using the underlying Iceberg table directly
from pyiceberg.expressions import EqualTo

def delete_window_by_key_with_retry(window_key: str, max_retries: int = 3) -> None:
    """Delete a window with automatic retry on concurrent modification."""
    for attempt in range(max_retries):
        try:
            # Reload the table from scratch to get fresh metadata
            fresh_store = build_window_store(use_rest_api_table=True)
            fresh_store.table.delete(EqualTo("window_key", window_key))
            print(f"✓ Deleted window: {window_key}")
            return
        except Exception as e:
            if "CommitFailedException" in str(type(e).__name__) and attempt < max_retries - 1:
                print(f"Retry {attempt + 1}/{max_retries} due to concurrent modification...")
                continue
            raise

# Example usage:
# delete_window_by_key_with_retry("your_window_key_here")